In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-26"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
15167,2024-10-26,Brasil Nbb,11:00,Brasília,Mogi,1.42,2.56,154.5,1.84,1.82,-6.5,2.02,1.67,lzvA8C2e,0.704225,0.390625,0.543478,0.549451,0.094850,0.6,0.4,NaN,NaN,271.868,127.352641,0.468436,267.932,150.659892,0.562306,317.060,363.574,123.12,457.50,0.0,0.0,0.0,0.0,0.405076,0.007728,0.134139,-3.48,-0.696,-0.603448,0.412304,0.2,-0.212304,-0.73,-0.146,-10.684932,0.261530,0.2,-0.061530
15168,2024-10-26,Brasil Nbb,11:00,Flamengo,Pinheiros,1.05,6.98,148.5,1.85,1.81,-15.5,1.92,1.75,xdK4UgPE,0.952381,0.143266,0.540541,0.552486,0.095647,0.8,0.2,NaN,NaN,99.148,25.221692,0.254384,158.236,60.842325,0.384504,100.384,171.398,75.60,140.00,0.0,0.0,0.0,0.0,1.044369,0.015456,0.065509,-0.50,-0.100,-0.500000,0.870017,0.9,0.029983,-0.43,-0.086,-69.534884,0.454179,0.5,0.045821
15169,2024-10-26,Brasil Nbb,17:00,São Paulo,Bauru,1.38,2.72,158.5,1.87,1.79,-6.5,1.88,1.78,MmxI6jX7,0.724638,0.367647,0.534759,0.558659,0.092285,0.8,0.6,NaN,NaN,152.376,40.938303,0.268666,145.076,33.708493,0.232351,178.682,171.840,218.12,167.04,0.0,0.0,0.0,0.0,0.462206,0.030912,0.038640,-0.25,-0.050,-7.600000,0.600717,0.4,-0.200717,2.14,0.428,4.018692,0.492390,0.5,0.007610
15170,2024-10-26,Espanha Acb,13:00,Baskonia,Joventut,1.28,3.75,168.5,1.82,1.92,-9.5,2.02,1.79,j38X5Vfd,0.781250,0.266667,0.549451,0.520833,0.047917,0.6,0.6,NaN,NaN,204.148,96.572239,0.473050,158.050,56.607271,0.358161,138.796,198.026,130.15,234.65,0.0,0.0,0.0,0.0,0.694455,0.037813,0.085372,0.97,0.194,1.443299,0.720010,0.8,0.079990,0.85,0.170,16.176471,0.429497,0.3,-0.129497
15171,2024-10-26,Espanha Acb,13:00,Basquet Girona,Murcia,2.36,1.60,160.5,1.84,1.90,2.5,1.98,1.77,Q1bP7Nk3,0.423729,0.625000,0.543478,0.526316,0.048729,0.2,0.8,NaN,NaN,276.870,223.719917,0.808032,189.644,69.701943,0.367541,118.058,242.218,75.40,287.04,0.0,0.0,0.0,0.0,0.271415,0.022688,0.079196,-1.92,-0.384,-3.541667,0.575181,0.4,-0.175181,-2.81,-0.562,-1.067616,0.486273,0.4,-0.086273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15400,2024-10-26,Rússia Superliga,13:00,Lokomotiv Kuban 2,Novosibirsk,2.17,1.64,147.5,1.88,1.82,-4.5,1.95,1.72,QsMPtfcH,0.460829,0.609756,0.531915,0.549451,0.070586,0.6,0.2,NaN,NaN,227.324,92.217997,0.405668,180.968,99.996903,0.552567,176.508,173.234,184.14,135.72,0.0,0.0,0.0,0.0,0.196728,0.022933,0.088629,-1.03,-0.206,-5.679612,0.000000,0.0,0.000000,-1.47,-0.294,-2.176871,0.000000,0.0,0.000000
15401,2024-10-26,Sérvia 1. Zls Feminino,10:00,Mega MIS F,Vojvodina 021 F,1.09,6.63,134.5,1.81,1.89,-17.5,1.88,1.78,QiqRgaRi,0.917431,0.150830,0.552486,0.529101,0.068261,0.0,0.4,NaN,NaN,143.464,70.800336,0.493506,142.268,55.787428,0.392129,0.000,140.560,157.38,121.91,0.0,0.0,0.0,0.0,1.014863,0.030578,0.038640,0.00,0.000,inf,0.000000,0.0,0.000000,-0.25,-0.050,-112.600000,0.000000,0.0,0.000000
15402,2024-10-26,Sérvia 1. Zls Feminino,11:30,ZKK Kraljevo F,Partizan F,1.11,5.17,137.5,1.80,1.86,-17.5,1.83,1.83,Aq1vWeZp,0.900901,0.193424,0.555556,0.537634,0.094324,0.0,0.4,NaN,NaN,135.686,67.176785,0.495090,193.598,196.858623,1.016842,0.000,627.040,253.68,93.48,0.0,0.0,0.0,0.0,0.914285,0.023184,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,20.00,4.000,1.042500,0.000000,0.0,0.000000
15403,2024-10-26,Sérvia 1. Zls Feminino,13:00,Proleter Zrenjanin F,Novosadska Novi Sad F,2.41,1.47,142.5,1.84,1.82,4.5,1.83

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Charlotte Hornets,Miami Heat,222.5,1.85,0.9920,Over
1,21:00,Eua Nba,Minnesota Timberwolves,Toronto Raptors,218.5,1.84,0.8672,Over
2,23:30,Eua Nba,Los Angeles Lakers,Sacramento Kings,230.5,1.86,0.9925,Over
